In [0]:
#mount the right drive an authenticate
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
from PIL import Image
import os
from skimage.transform import rescale
import cv2
import matplotlib.pyplot as plt
import gc
import numpy as np
import time
import datetime

from google.colab import output

#from IPython.core.debugger import set_trace

Image.MAX_IMAGE_PIXELS = 295470300

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
rescaling = 1/5

scale_percent = 20 # percent of original size

i=0

directories = []
every_image = []

destination = '/content/drive/My Drive/Seg_net/dataset/raw/'
os.chdir(destination)
every_image_already_processed = os.listdir('.')

origin = '/content/drive/My Drive/Dati Pascoli/'
os.chdir(origin)
every_image = os.listdir('.')

len(every_image)

#Extract folder which contains images only from western Trentino
for j in range(0, len(every_image)):
    name = every_image[j][0:3]
    if not((name == '027' or name == '028' or name == '044' or name == '045' or name == '061' or name == '062' or name == '063' or name == '082' or name == '083')):
        if not(every_image[j] in every_image_already_processed):
            if os.listdir(every_image[j]):
                directories.append(every_image[j])

cycle_time = []
print(len(directories))

for x in directories: 
    start_time = time.time()
    i = i + 1
    print('{} : {}'.format(i,x))
    name = x
    os.mkdir(destination + name)
    #RGB
    #go to folder in which data are stored
    print('RGB')
    os.chdir(origin + name)
    img_ref = Image.open(name + '_RGB.tif') #open image
    temp_arr = np.array(img_ref, np.float32) #convert in np array
    print('Input image shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    temp_arr = rescale(temp_arr, rescaling, multichannel=True) #upsample to 1m of ground resolution
    print('After treatement shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    temp_arr = temp_arr.astype('uint8') #set to the minimum necessary data type
    print('Saved shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    os.chdir(destination + name) #go to dest folder
    np.save(name + '_RGB.npy', temp_arr) #save data
    img = Image.fromarray(temp_arr, 'RGB')
    plt.imshow(img)
    plt.savefig('RGB')

    print('NIR')
    #NIR
    os.chdir(origin + name)
    img_ref = Image.open(name + '_NIR.tif')
    temp_arr = np.array(img_ref, np.float32)
    print('After treatement shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    temp_arr = rescale(temp_arr, rescaling, multichannel=True)
    print('After treatement shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    temp_arr = temp_arr.astype(np.uint8)
    print('Saved shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    os.chdir(destination + name)
    np.save(name + '_NIR.npy', temp_arr)
    img = Image.fromarray(temp_arr, 'RGB')
    plt.imshow(img)
    plt.savefig('NIR')

    print('TARE')
    #TARE
    os.chdir(origin + name)
    img_ref = Image.open('tare.tif')
    temp_arr = np.array(img_ref, np.uint16)
    print('After treatement shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    width = int(temp_arr.shape[1] * scale_percent / 100)
    height = int(temp_arr.shape[0] * scale_percent / 100)
    dim = (width, height)
    temp_arr = cv2.resize(temp_arr, dim, interpolation = cv2.INTER_NEAREST)
    print('After treatement shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    temp_arr = temp_arr.astype(np.int16)
    print('Saved shape {} and type {}'.format(temp_arr.shape,temp_arr.dtype))
    os.chdir(destination + name)
    np.save('tare.npy', temp_arr)
    temp_arr = temp_arr.astype(np.uint8)
    img = Image.fromarray(temp_arr, 'L')
    plt.imshow(img)
    plt.savefig('tare')
    gc.collect()

    elapsed_time = time.time() - start_time
    cycle_time.append(elapsed_time)
    it_mean_duration = np.mean(cycle_time)

    print('Iter take {} in mean. Subsampling process running from {} of approximately {}'.format(it_mean_duration,
                                            time.strftime('%H h %M m %S s', time.gmtime(it_mean_duration * i)),
                                            time.strftime('%H h %M m %S s', time.gmtime(it_mean_duration * len(directories)))))

output.eval_js('new Audio("https://www.soundhelix.com/examples/mp3/SoundHelix-Song-2.mp3").play()')



1
1 : 059134w
RGB
Input image shape (15301, 17451, 3) and type float32
After treatement shape (3060, 3490, 3) and type float32
Saved shape (3060, 3490, 3) and type uint8
NIR
After treatement shape (15301, 17451, 3) and type float32
After treatement shape (3060, 3490, 3) and type float32
Saved shape (3060, 3490, 3) and type uint8
TARE
After treatement shape (15301, 17451) and type uint16
After treatement shape (3060, 3490) and type uint16
Saved shape (3060, 3490) and type int16
Iter take 113.64908623695374 in mean. Subsampling process running from 00 h 01 m 53 s of approximately 00 h 01 m 53 s
